In [7]:
import leafmap
import requests
import pds4_tools
from ipyleaflet import Map, WMSLayer, LayersControl, Polygon, GeoData
import ipyleaflet
import pandas as pd
import io

# Step 1: Fetch MESSENGER MDIS image data from pds4

### Set and format parameters for request. 

Note: the below parameters were found by observing the HTTPS links below with a web json formatter and identifying which search parameters would be useful in the notebook. There are many more to choose from for each data set!

In [8]:
browse_4001 = 'https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:messenger_mdis_4001:browse/members'
prod_4001 = 'https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:messenger_mdis_4001:bdr_rdr/members'

browse_1001 = 'https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:messenger_mdis_dem_1001:browse::1.0/members'
prod_1001 = 'https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:messenger_mdis_dem_1001:elev::1.0/members'

north = 'cart:Bounding_Coordinates.cart:north_bounding_coordinate'
east = 'cart:Bounding_Coordinates.cart:east_bounding_coordinate'
south = 'cart:Bounding_Coordinates.cart:south_bounding_coordinate'
west = 'cart:Bounding_Coordinates.cart:west_bounding_coordinate'

fname_browse = ['ops:Data_File_Info.ops:file_ref', 'ref_lid_data']
fname_prod = [east,north,west,south,
              'lid',
              'ops:Label_File_Info.ops:file_size',
              'ops:Data_File_Info.ops:creation_date_time']

browse_params = dict(fields = ",".join(fname_browse))
prod_params = dict(fields = ",".join(fname_prod))

headers = dict(Accept="application/csv")

### Load the 4001 bundle and 1001 bundle

The function below requests the url for each product and sorts the data into a pandas dataframe for later use. 

In [9]:
def grab_products(browse_url,prod_url):
    browse_resp = requests.get(browse_url, params = browse_params, headers = headers)
    prod_resp = requests.get(prod_url, params = prod_params, headers = headers)

    browse_csv_stream = io.StringIO(browse_resp.content.decode('utf-8'))
    prod_csv_stream = io.StringIO(prod_resp.content.decode('utf-8'))

    browse_df_tot = pd.read_csv(browse_csv_stream)
    prod_df_tot = pd.read_csv(prod_csv_stream)

    #remove brackets - this is necessary due to a current bug in the PDS api. In the future this line may not be necessary. 
    prod_df_tot[[north,east,west,south]] = prod_df_tot[[north,east,west,south]].applymap(lambda x: float(x.strip('[]')), na_action = 'ignore')

    tot_df = browse_df_tot.join(prod_df_tot,lsuffix='_browse', rsuffix='_prod')
    tot_df = tot_df.dropna(subset=['cart:Bounding_Coordinates.cart:east_bounding_coordinate'])
    return tot_df

df_4001 = grab_products(browse_4001, prod_4001)
df_1001 = grab_products(browse_1001, prod_1001)


In [10]:
df_4001

,ops:Data_File_Info.ops:file_ref,ref_lid_data,cart:Bounding_Coordinates.cart:east_bounding_coordinate,cart:Bounding_Coordinates.cart:north_bounding_coordinate,cart:Bounding_Coordinates.cart:south_bounding_coordinate,cart:Bounding_Coordinates.cart:west_bounding_coordinate,lid,ops:Data_File_Info.ops:creation_date_time,ops:Label_File_Info.ops:file_size
0,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-89.996162,43.750000,22.500000,-135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:06:42Z, 2023-04-01T03:07:03Z]",[18190]
1,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-134.998015,65.000000,43.746094,180.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:06:17Z, 2023-04-01T03:06:41Z]",[18190]
2,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-144.000000,22.500000,0.000000,180.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:14:01Z, 2023-04-01T03:14:04Z]",[18184]
3,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-179.996162,43.750000,22.500000,135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:08:34Z, 2023-04-01T03:08:56Z]",[18189]
4,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-179.998015,65.000000,43.746094,135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:07:43Z, 2023-04-01T03:07:43Z]",[18189]
5,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,108.000000,0.000000,-22.500000,72.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:15:48Z, 2023-04-01T03:15:49Z]",[18178]
6,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,0.000000,0.000000,-22.500000,-36.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:10:51Z, 2023-04-01T03:12:20Z]",[18178]
7,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,90.003838,-22.500000,-43.750000,45.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:20:57Z, 2023-04-01T03:22:14Z]",[18188]
8,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,45.001985,-43.750000,-65.003906,-0.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:22:20Z, 2023-04-01T03:22:34Z]",[18189]
9,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-108.000000,22.500000,0.000000,-144.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:12:27Z, 2023-04-01T03:12:28Z]",[18185]


### Sort products and print data frames


In [4]:
df_1001 = df_1001[3:] #skip the first 2 products because we do not want to visualize these images

#Sort the values based on the east longitude coordinate - this is for the sake of layer visibility
df_1001 = df_1001.sort_values('cart:Bounding_Coordinates.cart:east_bounding_coordinate', ascending = False)
df_1001 = df_1001.reset_index(drop=True)

print('4001 Messenger Bundle Data Frame: ')
display(df_4001.head())
print('1001 Messenger Bundle Datra Frame: ')
display(df_1001.head())

4001 Messenger Bundle Data Frame: 


,ops:Data_File_Info.ops:file_ref,ref_lid_data,cart:Bounding_Coordinates.cart:east_bounding_coordinate,cart:Bounding_Coordinates.cart:north_bounding_coordinate,cart:Bounding_Coordinates.cart:south_bounding_coordinate,cart:Bounding_Coordinates.cart:west_bounding_coordinate,lid,ops:Data_File_Info.ops:creation_date_time,ops:Label_File_Info.ops:file_size
0,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-89.996162,43.75,22.500000,-135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:06:42Z, 2023-04-01T03:07:03Z]",[18190]
1,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-134.998015,65.00,43.746094,180.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:06:17Z, 2023-04-01T03:06:41Z]",[18190]
2,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-144.000000,22.50,0.000000,180.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:14:01Z, 2023-04-01T03:14:04Z]",[18184]
3,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-179.996162,43.75,22.500000,135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:08:34Z, 2023-04-01T03:08:56Z]",[18189]
4,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis...,-179.998015,65.00,43.746094,135.0,urn:nasa:pds:messenger_mdis_4001:bdr_rdr:mdis_...,"[2023-04-01T03:07:43Z, 2023-04-01T03:07:43Z]",[18189]


1001 Messenger Bundle Datra Frame: 


,ops:Data_File_Info.ops:file_ref,ref_lid_data,cart:Bounding_Coordinates.cart:east_bounding_coordinate,cart:Bounding_Coordinates.cart:north_bounding_coordinate,cart:Bounding_Coordinates.cart:south_bounding_coordinate,cart:Bounding_Coordinates.cart:west_bounding_coordinate,lid,ops:Data_File_Info.ops:creation_date_time,ops:Label_File_Info.ops:file_size
0,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_dem_1001:elev:msg...,156.864438,31.077864,30.651210,156.272546,urn:nasa:pds:messenger_mdis_dem_1001:elev:msgr...,"[2023-03-29T15:13:13Z, 2023-03-29T15:13:13Z]",[9721]
1,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_dem_1001:elev:msg...,155.135096,42.941071,42.136947,154.029018,urn:nasa:pds:messenger_mdis_dem_1001:elev:msgr...,"[2023-03-29T15:13:45Z, 2023-03-29T15:13:45Z]",[9718]
2,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_dem_1001:elev:msg...,154.885325,42.332266,41.776078,153.830279,urn:nasa:pds:messenger_mdis_dem_1001:elev:msgr...,"[2023-03-29T15:13:58Z, 2023-03-29T15:13:58Z]",[9749]
3,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_dem_1001:elev:msg...,154.706936,42.948582,42.339782,153.886344,urn:nasa:pds:messenger_mdis_dem_1001:elev:msgr...,"[2023-03-29T15:13:42Z, 2023-03-29T15:13:42Z]",[9722]
4,[https://asc-pds-messenger.s3.us-west-2.amazon...,[urn:nasa:pds:messenger_mdis_dem_1001:elev:msg...,154.495611,42.464234,42.239245,153.926804,urn:nasa:pds:messenger_mdis_dem_1001:elev:msgr...,"[2023-03-29T15:14:15Z, 2023-03-29T15:14:16Z]",[9718]


# Step 2: Visualize data on basemap

### Convert pandas dataframes into geodata geometry
The product information is viewable in an HTML widget using the geojson function in leafmap. The below code embeds the image URLS and product HTTPS into text that can be interacted with in the widget. 

In [5]:
from shapely import Point, LineString, Polygon
import geopandas as gpd

def geo_data_conversion(tot_df):
    
    obj_coords = []
    prod_names_html = []
    image_link_html = []
    file_date = []
    file_size = []
    
    for ind,row in tot_df.iterrows():
        prod_names = row['lid'].replace('[', '').replace(']', '')
        prod_names_html.append(f"<a href='https://pds.nasa.gov/api/search/1/products/{prod_names}'> View in Browser </a>")
        
        image_link = row['ops:Data_File_Info.ops:file_ref'].replace('[', '').replace(']', '')
        image_link_html.append(f"<a href='{image_link}'> View/Download </a>")
        
        file_size.append(row['ops:Label_File_Info.ops:file_size'])
        
        file_date.append(row['ops:Data_File_Info.ops:creation_date_time'])
        
        obj_coords.append([(row[east], row[south]),
                           (row[west], row[south]),
                           (row[west], row[north]),
                           (row[east], row[north])])
        
    polygons = [Polygon(coord) for coord in obj_coords] 

    gdf = gpd.GeoDataFrame({'geometry': polygons})
    gdf['Image'] = image_link_html
    gdf['Product Link'] = prod_names_html
    gdf['Creation_date'] = file_date
    gdf['File Size [Bytes]'] = file_size
    
    return gdf

gdf_4001 = geo_data_conversion(df_4001)
gdf_1001 = geo_data_conversion(df_1001)



### Instantiate basemap layers, convert geo data frames to geojson data and add them to the leafmap layer

In [6]:
wmsLayer = WMSLayer(
    url='https://planetarymaps.usgs.gov/cgi-bin/mapserv?map=/maps/mercury/mercury_simp_cyl.map',
    layers='MESSENGER',
    name='Messenger Basemap',
    crs=ipyleaflet.projections.EPSG4326
)

merc_map = leafmap.Map(layers=(wmsLayer, ), center=(0, 0), zoom=4, crs = ipyleaflet.projections.EPSG4326)
merc_map.add_control(LayersControl())


style1 = {
    "stroke": True,
    "color": "#0000ff",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.1,
}

style2 = {
    "stroke": True,
    "color": "#ff0000",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#ff0000",
    "fillOpacity": 0.1,
}

hover_style = {"fillOpacity": 0.7}

gdf4001_geojson = leafmap.gdf_to_geojson(gdf_4001, epsg="4326")
gdf1001_geojson = leafmap.gdf_to_geojson(gdf_1001, epsg="4326")

merc_map.add_geojson(gdf4001_geojson, layer_name="4001 Collection", style = style1, hover_style = hover_style)
merc_map.add_geojson(gdf1001_geojson, layer_name="1001 Collection", style = style2, hover_style = hover_style)

# print('urn:nasa:pds:messenger_mdis_4001:bdr_rdr:')
merc_map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…